# Solving Large Stiff Equations

## Contents

- [Definition of the Brusselator Equation](#definition_of_the_brusselator_equation)
- [Choosing Jacobian Types](#choosing_jacobian_types)

This tutorial is for getting into the extra features for solving large stiff ordinary
differential equations efficiently. Solving stiff ordinary
differential equations requires specializing the linear solver on properties of
the Jacobian in order to cut down on the $\mathcal{O}(n^3)$ linear solve and
the $\mathcal{O}(n^2)$ back-solves. Note that these same functions and
controls also extend to stiff SDEs, DDEs, DAEs, etc. This tutorial is for large-scale
models, such as those derived for semi-discretizations of partial differential
equations (PDEs). For example, we will use the stiff Brusselator partial
differential equation (BRUSS).

## Definition of the Brusselator Equation <a id="definition_of_the_brusselator_equation" />

The Brusselator PDE is defined on a unit square periodic domain as follows:

```math
\begin{align}
\frac{\partial U}{\partial t} &= 1 + U^2V - 4.4U + \alpha \nabla^2 U + f(x, y, t),\\
\frac{\partial V}{\partial t} &= 3.4U - U^2V + \alpha \nabla^2 V,
\end{align}
```

where

```math
f(x, y, t) = \begin{cases}
5 & \quad \text{if } (x-0.3)^2+(y-0.6)^2 ≤ 0.1^2 \text{ and } t ≥ 1.1\\
0 & \quad \text{else}
\end{cases}, 
```

and

$$\nabla^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}$$

is the two dimensional Laplacian operator. The above equations are to be solved for a time interval $t \in [0, 11.5]$ subject to the initial conditions

```math
\begin{align}
U(x, y, 0) &= 22 \cdot (y(1-y))^{3/2} \\
V(x, y, 0) &= 27 \cdot (x(1-x))^{3/2}
\end{align}
```

and the periodic boundary conditions

```math
\begin{align}
U(x+1,y,t) &= U(x,y,t) \\
V(x,y+1,t) &= V(x,y,t).
\end{align}
```

To solve this PDE, we will discretize it into a system of ODEs with the finite
difference method. We discretize the unit square domain with `N` grid points in each direction.
`U[i,j]` and `V[i,j]` then represent the value of the discretized field at a given point in time, i.e.

```julia
U[i,j] = U(i*dx,j*dy)
V[i,j] = V(i*dx,j*dy)
```

where `dx = dy = 1/N`. To implement our ODE system, we collect both `U` and `V` in a single array `u` of size `(N,N,2)` with `u[i,j,1] = U[i,j]` and `u[i,j,2] = V[i,j]`. This approach can be easily generalized to PDEs with larger number of field variables.

Using a three-point stencil, the Laplacian operator discretizes into a tridiagonal matrix with elements `[1 -2 1]` and a `1` in the top, bottom, left, and right corners coming from the periodic boundary conditions. The nonlinear terms are implemented pointwise in a straightforward manner.

The resulting `ODEProblem` definition is:

In [39]:
using LinearAlgebra

Nx = Ny = 99

dx = 1/(Nx+1)
dy = 1/(Ny+1)
x_vec = dx:dx:1-dx
y_vec = dy:dy:1-dy

u = zeros(Nx, Ny, 2)
Ax = Tridiagonal([1.0 for i in 1:(Nx - 1)], [-2.0 for i in 1:Nx], [1.0 for i in 1:(Nx - 1)])
Ay = copy(Ax);

function f(x,y,t)
    if (x-0.3)^2 + (y-0.6)^2 <= 0.1^2 && t >= 1.1
        5
    else
        0
    end
end

# Initial conditions
u0 = copy(u)
grid = copy(u)
for i in 1:Nx, j in 1:Ny
    u0[i,j,1] = 22 * (y_vec[i]*(1-y_vec[i]))^(3/2)
    u0[i,j,2] = 27 * (x_vec[i]*(1-x_vec[i]))^(3/2)
    grid[i,j,1] = x_vec[i]
    grid[i,j,2] = y_vec[j]
end

# Boundary conditions

In [40]:
function Bruss_ODE(du, u, p, t)

    Ax, Ay, grid, α, f = p

    U = u[:,:,1]
    V = u[:,:,1]
    DU = Ax*U
    DV = Ay*V

    du[:,:,1] = @. 1 + U^2*V - 4.4*U + α*DU + f.(grid[:,:,1], grid[:,:,2], t)
    du[:,:,2] = @. 3.4*U - U^2*V + α*DV

end

Bruss_ODE (generic function with 1 method)

In [41]:
function Bruss_ODE(du, u, p, t)

    XY, α, f = p

    U = u[:,:,1]
    V = u[:,:,1]

    for i in 2:Nx-1
        for j in 2:Ny-1
            du[:,:,1] = 1 + U[i,j]^2 * V[i,j] - 4.4*U[i,j] + α * (U[i-1,j] + U[i+1,j] + U[i,j-1] + U[i,j+1] - 4U[i,j]) + f.(XY[i,j,1], XY[i,j,2], t)
            du[:,:,2] = 3.4*U[i,j] - U[i,j]^2*V[i,j] + α * (V[i-1,j] + V[i+1,j] + V[i,j-1] + V[i,j+1] - 4V[i,j])
        end
    end

    j = 1
    for i in 2:Nx-1
        du[i,j,1] = 1 + U[i,j]^2 * V[i,j] - 4.4*U[i,j] + α * (U[i-1,j] + U[i+1,j] + U[i,end] + U[i,j+1] - 4U[i,j]) + f.(XY[i,j,1], XY[i,j,2], t)
        du[i,j,2] = 3.4*U[i,j] - U[i,j]^2*V[i,j] + α * (V[i-1,j] + V[i+1,j] + V[i,end] + V[i,j+1] - 4V[i,j])
    end
    i = 1
    for j in 2:Ny-1
        du[i,j,1] = 1 + U[i,j]^2 * V[i,j] - 4.4*U[i,j] + α * (U[end,j] + U[i+1,j] + U[i,j-1] + U[i,j+1] - 4U[i,j]) + f.(XY[i,j,1], XY[i,j,2], t)
        du[i,j,2] = 3.4*U[i,j] - U[i,j]^2*V[i,j] + α * (V[end,j] + V[i+1,j] + V[i,j-1] + V[i,j+1] - 4V[i,j])
    end
    j =
    for i in 2:Nx-1
        du[i,j,1] = 1 + U[i,j]^2 * V[i,j] - 4.4*U[i,j] + α * (U[i-1,j] + U[i+1,j] + U[i,end] + U[i,j+1] - 4U[i,j]) + f.(XY[i,j,1], XY[i,j,2], t)
        du[i,j,2] = 3.4*U[i,j] - U[i,j]^2*V[i,j] + α * (V[i-1,j] + V[i+1,j] + V[i,end] + V[i,j+1] - 4V[i,j])
    end
    i = 1
    for j in 2:Ny-1
        du[i,j,1] = 1 + U[i,j]^2 * V[i,j] - 4.4*U[i,j] + α * (U[end,j] + U[i+1,j] + U[i,j-1] + U[i,j+1] - 4U[i,j]) + f.(XY[i,j,1], XY[i,j,2], t)
        du[i,j,2] = 3.4*U[i,j] - U[i,j]^2*V[i,j] + α * (V[end,j] + V[i+1,j] + V[i,j-1] + V[i,j+1] - 4V[i,j])
    end

end

Bruss_ODE (generic function with 1 method)

In [42]:
using DifferentialEquations, LinearAlgebra, SparseArrays

const N = 32
const xyd_brusselator = range(0, stop = 1, length = N)

0.0:0.03225806451612903:1.0

In [43]:
brusselator_f(x, y, t) = (((x - 0.3)^2 + (y - 0.6)^2) <= 0.1^2) * (t >= 1.1) * 5.0

brusselator_f (generic function with 1 method)

> Notice: this is cool. They use a simple `<=` condition in a mathemathical expression to get the value when it evaluates to `True`, and $0$ when it evaluates to `False`.

In [ ]:
limit(a, N) = a == N + 1 ? 1 : a == 0 ? N : a

In [ ]:
function brusselator_2d_loop(du, u, p, t)
    A, B, alpha, dx = p
    alpha = alpha / dx^2
    @inbounds for I in CartesianIndices((N, N))
        i, j = Tuple(I)
        x, y = xyd_brusselator[I[1]], xyd_brusselator[I[2]]
        ip1, im1, jp1, jm1 = limit(i + 1, N), limit(i - 1, N), limit(j + 1, N),
        limit(j - 1, N)
        du[i, j, 1] = alpha * (u[im1, j, 1] + u[ip1, j, 1] + u[i, jp1, 1] + u[i, jm1, 1] -
                       4u[i, j, 1]) +
                      B + u[i, j, 1]^2 * u[i, j, 2] - (A + 1) * u[i, j, 1] +
                      brusselator_f(x, y, t)
        du[i, j, 2] = alpha * (u[im1, j, 2] + u[ip1, j, 2] + u[i, jp1, 2] + u[i, jm1, 2] -
                       4u[i, j, 2]) +
                      A * u[i, j, 1] - u[i, j, 1]^2 * u[i, j, 2]
    end
end

In [ ]:
p = (3.4, 1.0, 10.0, step(xyd_brusselator))

function init_brusselator_2d(xyd)
    N = length(xyd)
    u = zeros(N, N, 2)
    for I in CartesianIndices((N, N))
        x = xyd[I[1]]
        y = xyd[I[2]]
        u[I, 1] = 22 * (y * (1 - y))^(3 / 2)
        u[I, 2] = 27 * (x * (1 - x))^(3 / 2)
    end
    u
end
u0 = init_brusselator_2d(xyd_brusselator)
prob_ode_brusselator_2d = ODEProblem(brusselator_2d_loop, u0, (0.0, 11.5), p)

In [ ]:
using DifferentialEquations, LinearAlgebra, SparseArrays

const N = 32
const xyd_brusselator = range(0, stop = 1, length = N)

brusselator_f(x, y, t) = (((x - 0.3)^2 + (y - 0.6)^2) <= 0.1^2) * (t >= 1.1) * 5.0
limit(a, N) = a == N + 1 ? 1 : a == 0 ? N : a

function brusselator_2d_loop(du, u, p, t)
    A, B, alpha, dx = p
    alpha = alpha / dx^2
    @inbounds for I in CartesianIndices((N, N))
        i, j = Tuple(I)
        x, y = xyd_brusselator[I[1]], xyd_brusselator[I[2]]
        ip1, im1, jp1, jm1 = limit(i + 1, N), limit(i - 1, N), limit(j + 1, N),
        limit(j - 1, N)
        du[i, j, 1] = alpha * (u[im1, j, 1] + u[ip1, j, 1] + u[i, jp1, 1] + u[i, jm1, 1] -
                       4u[i, j, 1]) +
                      B + u[i, j, 1]^2 * u[i, j, 2] - (A + 1) * u[i, j, 1] +
                      brusselator_f(x, y, t)
        du[i, j, 2] = alpha * (u[im1, j, 2] + u[ip1, j, 2] + u[i, jp1, 2] + u[i, jm1, 2] -
                       4u[i, j, 2]) +
                      A * u[i, j, 1] - u[i, j, 1]^2 * u[i, j, 2]
    end
end
p = (3.4, 1.0, 10.0, step(xyd_brusselator))

function init_brusselator_2d(xyd)
    N = length(xyd)
    u = zeros(N, N, 2)
    for I in CartesianIndices((N, N))
        x = xyd[I[1]]
        y = xyd[I[2]]
        u[I, 1] = 22 * (y * (1 - y))^(3 / 2)
        u[I, 2] = 27 * (x * (1 - x))^(3 / 2)
    end
    u
end

u0 = init_brusselator_2d(xyd_brusselator)
prob_ode_brusselator_2d = ODEProblem(brusselator_2d_loop, u0, (0.0, 11.5), p)

## Choosing Jacobian Types <a id="choosing_jacobian_types" />

When one is using an implicit or semi-implicit differential equation solver,
the Jacobian must be built at many iterations, and this can be one of the most
expensive steps. There are two pieces that must be optimized in order to reach
maximal efficiency when solving stiff equations: the sparsity pattern and the
construction of the Jacobian. The construction is filling the matrix
`J` with values, while the sparsity pattern is what `J` to use.

The sparsity pattern is given by a prototype matrix, the `jac_prototype`, which
will be copied to be used as `J`. The default is for `J` to be a `Matrix`,
i.e. a dense matrix. However, if you know the sparsity of your problem, then
you can pass a different matrix type. For example, a `SparseMatrixCSC` will
give a sparse matrix. Other sparse matrix types include:

  - Bidiagonal
  - Tridiagonal
  - SymTridiagonal
  - BandedMatrix ([BandedMatrices.jl](https://github.com/JuliaLinearAlgebra/BandedMatrices.jl))
  - BlockBandedMatrix ([BlockBandedMatrices.jl](https://github.com/JuliaLinearAlgebra/BlockBandedMatrices.jl))

DifferentialEquations.jl will internally use this matrix
type, making the factorizations faster by using the specialized forms.


## Declaring a Sparse Jacobian with Automatic Sparsity Detection

Jacobian sparsity is declared by the `jac_prototype` argument in the `ODEFunction`.
Note that you should only do this if the sparsity is high, for example, 0.1%
of the matrix is non-zeros, otherwise the overhead of sparse matrices can be higher
than the gains from sparse differentiation!

[ADTypes.jl](https://github.com/SciML/ADTypes.jl) provides a [common interface for automatic sparsity detection](https://sciml.github.io/ADTypes.jl/stable/#Sparsity-detector)
via its function `jacobian_sparsity`.
This function can be called using sparsity detectors from [SparseConnectivityTracer.jl](https://github.com/adrhill/SparseConnectivityTracer.jl)
or [Symbolics.jl](https://github.com/JuliaSymbolics/Symbolics.jl).

We can give an example `du` and `u` and call `jacobian_sparsity` on our function with the example arguments,
and it will kick out a sparse matrix with our pattern, that we can turn into our `jac_prototype`.

Let's try SparseConnectivityTracer's [`TracerSparsityDetector`](https://adrianhill.de/SparseConnectivityTracer.jl/stable/user/api/#SparseConnectivityTracer.TracerSparsityDetector):

In [ ]:
using SparseConnectivityTracer, ADTypes

detector = TracerSparsityDetector()
du0 = copy(u0)
jac_sparsity = ADTypes.jacobian_sparsity(
    (du, u) -> brusselator_2d_loop(du, u, p, 0.0), du0, u0, detector)

Using a different backend for sparsity detection just requires swapping out the detector,
e.g. for Symbolics' [`SymbolicsSparsityDetector`](https://docs.sciml.ai/Symbolics/stable/manual/sparsity_detection/#Symbolics.SymbolicsSparsityDetector).

Notice that Julia gives a nice print out of the sparsity pattern.
That's neat, and would be tedious to build by hand!
Now we just pass it to the `ODEFunction` like as before:

In [ ]:
f = ODEFunction(brusselator_2d_loop; jac_prototype = float.(jac_sparsity))

In [ ]:
prob_ode_brusselator_2d_sparse = ODEProblem(f, u0, (0.0, 11.5), p)